TODO: 1. For a given test set (word,sentence,document) get the ranking of concepts
      2. Find similarity between two words, sentence and document (using cosine product)

In [1]:
from gensim import *

import json 
data = [json.loads(_) for _ in open('wiki-100k-sample.json')]
from gensim.parsing import preprocess_string
corpus_data = []
for article in data[:100]:
    document = preprocess_string(str(article['section_texts']))
    corpus_data.append(document)
    


In [2]:
## inbuilt tfidf
dictionary = corpora.Dictionary(corpus_data)
dictionary.save('/tmp/corpus_data.dict') 

raw_corpus = [dictionary.doc2bow(t) for t in corpus_data]

corpora.MmCorpus.serialize('/tmp/corpus_data.mm', raw_corpus) # store to disk

dictionary = corpora.Dictionary.load('/tmp/corpus_data.dict')

corpus = corpora.MmCorpus('/tmp/corpus_data.mm')



In [3]:
import numpy as np

In [4]:
#Matrix initialization
b = np.zeros([corpus.num_docs,corpus.num_terms])

In [5]:
from gensim import sklearn_api
sk_tfidf = sklearn_api.tfidf.TfIdfTransformer()

In [6]:
a = sk_tfidf.fit_transform(corpus)


In [7]:
a

[[(0, 0.009553918039913819),
  (1, 0.04469995707267615),
  (2, 0.005050699521883645),
  (3, 0.018271765923075643),
  (4, 0.01116266717319636),
  (5, 0.00955391803991382),
  (6, 0.008412492364897991),
  (7, 0.006007549426170123),
  (8, 0.0127714163064789),
  (9, 0.00809490983061729),
  (10, 0.045208918815837926),
  (11, 0.036543531846151285),
  (12, 0.006007549426170123),
  (13, 0.015521591114777269),
  (14, 0.018271765923075643),
  (15, 0.00809490983061729),
  (16, 0.00809490983061729),
  (17, 0.014542133379764323),
  (18, 0.010021241498180532),
  (19, 0.0383142489194367),
  (20, 0.009553918039913819),
  (21, 0.00638570815323945),
  (22, 0.0127714163064789),
  (23, 0.006192125196899475),
  (24, 0.0056623175565996215),
  (25, 0.006192125196899475),
  (26, 0.013029498471487017),
  (27, 0.00707583088471282),
  (28, 0.015521591114777269),
  (29, 0.031043182229554538),
  (30, 0.018271765923075643),
  (31, 0.012672839546697477),
  (32, 0.018271765923075643),
  (33, 0.018271765923075643),
  (

In [8]:
for i in range(len(a)):
    for j in a[i]:
        b[i,j[0]] = j[1]
matrix_T = b

In [9]:
matrix_T

array([[ 0.00955392,  0.04469996,  0.0050507 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.00178742,  0.00141374, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.00315909,  0.        ,  0.00334012, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.01364905,
         0.01364905,  0.01364905]])

In [10]:
import pandas as pd
esa = pd.DataFrame(matrix_T,index=[i for i in range(np.shape(matrix_T)[0])],columns=[dictionary.get(i) for i in range(np.shape(matrix_T)[1])])
esa = esa.transpose()
esa.head()


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
shop,0.009554,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.008132,0.000000,0.000000,0.0,0.003159,0.0,0.0
code,0.044700,0.0,0.001787,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.006515,0.003425,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
consider,0.005051,0.0,0.001414,0.0,0.0,0.0,0.0,0.0,0.0,0.008764,...,0.000000,0.002577,0.000000,0.002150,0.004548,0.004181,0.0,0.003340,0.0,0.0
consum,0.018272,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.002304,0.000000,0.001633,0.007777,0.000000,0.007563,0.0,0.000000,0.0,0.0
interchang,0.011163,0.0,0.003125,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [11]:
test_1 = "shop"
test_2 = "code"
test_3 = "consider consum interchang"

test =[test_1,test_2,test_3]

def get_word_rank(string):
    words = string.split()
    vector = esa.loc[words[0]].as_matrix()
    for i in range(1,len(words)):
        vector = vector + esa.loc[words[i]].as_matrix()
    return vector/len(words)

def get_similarity(t1,t2):
    doc1 = get_word_rank(t1)
    doc2 = get_word_rank(t2)
    sim = np.dot(doc1,doc2)/np.sqrt((np.dot(doc1,doc1)*np.dot(doc2,doc2)))
    return sim


for i in test:
    for j in test:
        print "doc1:",i
        print "doc2:", j
        print get_similarity(i,j)
        print
    


doc1: shop
doc2: shop
1.0

doc1: shop
doc2: code
0.247207511596

doc1: shop
doc2: consider consum interchang
0.447798096915

doc1: code
doc2: shop
0.247207511596

doc1: code
doc2: code
1.0

doc1: code
doc2: consider consum interchang
0.34554380735

doc1: consider consum interchang
doc2: shop
0.447798096915

doc1: consider consum interchang
doc2: code
0.34554380735

doc1: consider consum interchang
doc2: consider consum interchang
1.0



## manual tfidf 

In [12]:

mydoclist = corpus_data
from collections import Counter

for doc in mydoclist[:10]:
    tf = Counter()
    for word in doc:
        tf[word] +=1
    print tf.items()

[(u'slope', 1), (u'code', 7), (u'consider', 1), (u'consum', 2), (u'interchang', 1), (u'suction', 1), (u'edg', 1), (u'privat', 1), (u'decim', 1), (u'environment', 1), (u'program', 10), (u'usnc', 2), (u'digit', 1), (u'stanton', 1), (u'fals', 1), (u'electron', 1), (u'mechan', 1), (u'govern', 4), (u'relev', 1), (u'din', 3), (u'headquart', 1), (u'prevent', 1), (u'cap', 1), (u'encourag', 1), (u'budget', 2), (u'rate', 1), (u'design', 4), (u'nthe', 5), (u'quicken', 1), (u'electrotechn', 2), (u'accreditor', 1), (u'current', 4), (u'nhad', 1), (u'spri', 1), (u'new', 2), (u'appeal', 1), (u'public', 2), (u'impact', 1), (u'iest', 1), (u'bodi', 5), (u'china', 1), (u'valu', 1), (u'search', 1), (u'invit', 1), (u'worldwid', 2), (u'chang', 3), (u'commonli', 1), (u'bolt', 1), (u'depart', 2), (u'approxim', 1), (u'total', 1), (u'establish', 1), (u'market', 2), (u'clifford', 1), (u'fee', 2), (u'astm', 3), (u'internation', 1), (u'petroleum', 1), (u'advoc', 1), (u'taken', 1), (u'peopl', 1), (u'notif', 1), (u'g

In [13]:
import string #allows for format()
mydoclist = corpus_data   
def build_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc])
    return lexicon

def tf(term, document):
  return freq(term, document)

def freq(term, document):
  return document.count(term)

vocabulary = build_lexicon(mydoclist[:10])

doc_term_matrix = []
# print 'Our vocabulary vector is [' + ', '.join(list(vocabulary)) + ']'
for doc in mydoclist[:10]:

    tf_vector = [tf(word, doc) for word in vocabulary]
    tf_vector_string = ', '.join(format(freq, 'd') for freq in tf_vector)
#     print 'The tf vector for Document %d is [%s]' % ((mydoclist.index(doc)+1), tf_vector_string)
    doc_term_matrix.append(tf_vector)
    
    # here's a test: why did I wrap mydoclist.index(doc)+1 in parens?  it returns an int...
    # try it!  type(mydoclist.index(doc) + 1)

print 'All combined, here is our master document term matrix: '
print doc_term_matrix

All combined, here is our master document term matrix: 
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 

In [14]:
import scipy

In [15]:
sparse =scipy.sparse.coo_matrix(doc_term_matrix)

In [16]:
(sparse/3.0).toarray()

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.33333333,  0.        ,  1.66666667, ...,  0.66666667,
         1.        ,  0.33333333],
       ..., 
       [ 0.        ,  0.66666667,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [17]:
sparse.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 5, ..., 2, 3, 1],
       ..., 
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
# esa = pd.DataFrame(sparse.toarray(),index=[i for i in range(np.shape(matrix_T)[0])],columns=[dictionary.get(i) for i in range(np.shape(matrix_T)[1])])
esa = pd.DataFrame(sparse.toarray())

In [19]:
esa

,0,1,2,3,4,5,6,7,8,9,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,13,0,0,0
2,1,0,5,1,0,1,0,2,1,0,...,0,1,2,0,0,0,1,2,3,1
3,0,0,0,0,0,0,1,0,0,0,...,1,0,0,2,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9,0,0,0,0,0,0,0,0,1,3,...,0,0,2,0,0,0,0,0,0,0


In [20]:
def numDocsContaining(word, doclist):
    doccount = 0
    for doc in doclist:
        if freq(word, doc) > 0:
            doccount +=1
    return doccount 

def idf(word, doclist):
    n_samples = len(doclist)
    df = numDocsContaining(word, doclist)
    return np.log(n_samples / 1+df)

my_idf_vector = [idf(word, mydoclist[:10]) for word in vocabulary]

In [21]:
def build_idf_matrix(idf_vector):
    idf_mat = np.zeros((len(idf_vector), len(idf_vector)))
    np.fill_diagonal(idf_mat, idf_vector)
    return idf_mat

my_idf_matrix = build_idf_matrix(my_idf_vector)

In [22]:
my_idf_matrix

array([[ 2.39789527,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.39789527,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.39789527, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  2.39789527,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         2.39789527,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  2.39789527]])

In [23]:


def l2_normalizer(vec):
    denom = np.sum([el**2 for el in vec])
    return [(el / np.sqrt(denom)) for el in vec]


In [24]:
doc_term_matrix_tfidf = []

#performing tf-idf matrix multiplication
for tf_vector in doc_term_matrix:
    doc_term_matrix_tfidf.append(np.dot(tf_vector, my_idf_matrix))

#normalizing
doc_term_matrix_tfidf_l2 = []
for tf_vector in doc_term_matrix_tfidf:
    doc_term_matrix_tfidf_l2.append(l2_normalizer(tf_vector))
    
doc_term_matrix_tfidf_l2

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0086659645283313769,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0089451087229471555,
  0.0083625175128800196,
  0.0,
  0.0083625175128800196,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.037776657703732355,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.018407111759011073,
  0.0,
  0.0,
  0.0,
  0.0,
  0.04940331730948723,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.019338477379527666,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0089451087229471555,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

## second order implementation

In [25]:
### sample links data 

links = { 1: [2,3,4,5],2:[3,5],3:[],4:[2],5:[1]}

In [26]:
esa2 = esa
alpha = 0.5
for i in links.keys():
    for j in links[i]:
        esa2[i] = esa[i] + alpha*esa[j]

In [27]:
esa2

,0,1,2,3,4,5,6,7,8,9,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922
0,0,0.5,0.5,0,0.25,1.25,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0,0.0,0,0.00,0.00,0,0,1,0,...,0,0,0,0,1,1,13,0,0,0
2,1,3.5,6.0,1,3.00,2.75,0,2,1,0,...,0,1,2,0,0,0,1,2,3,1
3,0,0.0,0.0,0,0.00,0.00,1,0,0,0,...,1,0,0,2,0,0,0,0,0,0
4,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,3.0,0.0,0,2.00,1.50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0.0,0.0,0,0.00,0.00,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9,0,0.0,0.0,0,0.00,0.00,0,0,1,3,...,0,0,2,0,0,0,0,0,0,0
